In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('Cleaning_Data.csv')
df.head()

,Unnamed: 0,Column,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pct_tl_nvr_dlq,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,0,0,20000,20000,20000,60 months,17.19%,499.10,C,6 years,...,13,33,4,15,5,98.0,75824,31546,33800,21524
1,1,1,21225,21225,21225,60 months,14.74%,502.05,C,10+ years,...,7,7,6,14,2,100.0,747075,209426,53500,128175
2,2,2,5000,5000,5000,36 months,17.97%,180.69,D,NaN,...,6,9,6,8,1,66.7,255738,31615,9400,39938
3,3,3,20000,20000,20000,36 months,8.19%,628.49,A,1 year,...,14,27,9,20,3,95.7,448069,84744,49400,105180
4,4,4,13500,13500,13500,36 months,7.56%,420.31,A,8 years,...,9,11,6,9,1,100.0,40200,9932,25100,0


In [5]:
# Loan status summary 
df.loan_status.value_counts()

Current               110918
Fully Paid              3608
Late (31-120 days)       468
In Grace Period          327
Late (16-30 days)        256
Charged Off               98
Name: loan_status, dtype: int64

In [6]:
# remove all current loans 
df = df[df.loan_status != 'Current']
df.shape

(4757, 71)

In [7]:
# change loan status to numeric where 1 are fully paid loans and 0 charged off loans
di = {"Fully Paid":1, "Charged Off":0, "Late (31-120 days)":0, "In Grace Period":0, "Late (16-30 days)":0}  
df= df.replace({"loan_status": di})
df["loan_status"].value_counts()
newMapping = {1:"Fully Paid", 0:"Charged Off"}
df= df.replace({"loan_status": newMapping})

In [8]:
# change interest rate to numeric 
df['int_rate'] = df['int_rate'].str.rstrip('%').astype('float')

In [9]:
# change employment length to numeric
map = {"n/a":0, "< 1 ":1, "1 ":1, "2 ":2, "3 ":3, "4 ":4, "5 ":5, "6 ":6, "7 ":7, "8 ":8, "9 ":9, "10+ ":10}
df.emp_length = df.emp_length.str.replace("years","")
df.emp_length = df.emp_length.str.replace("year","")
df.emp_length.replace(map, inplace=True)
df.emp_length.value_counts()

10.0    1500
1.0      876
2.0      387
3.0      360
5.0      295
4.0      251
6.0      197
7.0      180
8.0      147
9.0       95
Name: emp_length, dtype: int64

In [10]:
# change grade to numeric
df['grade'] = df['grade'].map({'A':7,'B':6,'C':5,'D':4,'E':3,'F':2,'G':1})

In [11]:
# change home ownership to numeric 
df["home_ownership"] = df["home_ownership"].map({"MORTGAGE":6,"RENT":5,"OWN":4,"OTHER":3,"NONE":2,"ANY":1})

In [12]:
df.head()

,Unnamed: 0,Column,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pct_tl_nvr_dlq,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
7,7,7,10000,10000,10000,60 months,12.40,224.48,6,NaN,...,4,7,2,7,0,100.0,176439,37523,17000,57439
51,51,51,15000,15000,15000,60 months,17.97,380.66,4,9.0,...,7,7,5,8,4,100.0,37222,23300,16400,14822
80,80,80,35000,35000,35000,60 months,13.08,797.80,6,1.0,...,10,11,6,14,2,100.0,399376,131350,32700,130876
135,135,135,17500,17500,17500,36 months,17.19,625.58,5,5.0,...,6,6,5,12,0,92.6,131463,141781,17400,113163
146,146,146,13750,13750,13750,36 months,28.80,574.71,4,NaN,...,6,7,6,6,2,87.5,19200,16814,15400,0


In [13]:
# drop numerical columns if 20% of it or more are zeros
df = df.loc[:, (df==0).mean() < .25]

# drop columns if almost 25% of it or more are null values
df = df.dropna(thresh = 3757, axis=1)# 25% of the data 

# drop columns with a redundant value 
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df = df.drop(cols_to_drop, axis=1)

#Replace empty/null values with a space
Replace =df.fillna(" ")
Replace.isnull().sum()

Unnamed: 0                    0
Column                        0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installment                   0
grade                         0
emp_length                    0
home_ownership                0
annual_inc                    0
verification_status           0
issue_d                       0
loan_status                   0
purpose                       0
addr_state                    0
dti                           0
earliest_cr_line              0
open_acc                      0
revol_bal                     0
revol_util                    0
total_acc                     0
initial_list_status           0
total_pymnt                   0
total_pymnt_inv               0
total_rec_prncp               0
total_rec_int                 0
last_pymnt_d                  0
last_pymnt_amnt               0
                             ..
il_util 

In [ ]:
#final_dataa.to_csv('finalData.csv')

In [14]:
df.head()

,Unnamed: 0,Column,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pct_tl_nvr_dlq,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
7,7,7,10000,10000,10000,60 months,12.40,224.48,6,NaN,...,4,7,2,7,0,100.0,176439,37523,17000,57439
51,51,51,15000,15000,15000,60 months,17.97,380.66,4,9.0,...,7,7,5,8,4,100.0,37222,23300,16400,14822
80,80,80,35000,35000,35000,60 months,13.08,797.80,6,1.0,...,10,11,6,14,2,100.0,399376,131350,32700,130876
135,135,135,17500,17500,17500,36 months,17.19,625.58,5,5.0,...,6,6,5,12,0,92.6,131463,141781,17400,113163
146,146,146,13750,13750,13750,36 months,28.80,574.71,4,NaN,...,6,7,6,6,2,87.5,19200,16814,15400,0


In [15]:
df.to_csv('cleanSplit.csv')

In [16]:
df1 = pd.read_csv('cleanSplit.csv')
df1.head()

,Unnamed: 0,Unnamed: 0.1,Column,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_op_past_12m,pct_tl_nvr_dlq,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,7,7,7,10000,10000,10000,60 months,12.40,224.48,6,...,4,7,2,7,0,100.0,176439,37523,17000,57439
1,51,51,51,15000,15000,15000,60 months,17.97,380.66,4,...,7,7,5,8,4,100.0,37222,23300,16400,14822
2,80,80,80,35000,35000,35000,60 months,13.08,797.80,6,...,10,11,6,14,2,100.0,399376,131350,32700,130876
3,135,135,135,17500,17500,17500,36 months,17.19,625.58,5,...,6,6,5,12,0,92.6,131463,141781,17400,113163
4,146,146,146,13750,13750,13750,36 months,28.80,574.71,4,...,6,7,6,6,2,87.5,19200,16814,15400,0


In [21]:
# drop columns
df1.drop(['Unnamed: 0', 'Unnamed: 0.1','Column', 'term','open_acc','total_pymnt','total_pymnt_inv',
         'home_ownership','verification_status','issue_d','total_rec_prncp','last_pymnt_d',
         'last_credit_pull_d','application_type','tot_cur_bal','open_act_il','mths_since_rcnt_il',
         'total_bal_il','il_util','mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mths_since_recent_bc',
         'mths_since_recent_inq','num_actv_bc_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl',
         'num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_op_past_12m','pct_tl_nvr_dlq','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit',
         'total_il_high_credit_limit','dti','revol_bal','total_acc','initial_list_status','open_rv_24m','max_bal_bc',
        'all_util','total_rev_hi_lim','acc_open_past_24mths','bc_open_to_buy','bc_util'], axis=1, inplace=True)

In [22]:
df1.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,emp_length,annual_inc,loan_status,purpose,addr_state,earliest_cr_line,revol_util,total_rec_int,last_pymnt_amnt,avg_cur_bal,num_actv_rev_tl
0,10000,10000,10000,12.40,224.48,6,NaN,38000.0,Fully Paid,credit_card,GA,Feb-97,49%,149.96,10163.74,19266.0,2
1,15000,15000,15000,17.97,380.66,4,9.0,45000.0,Fully Paid,debt_consolidation,FL,Mar-07,48%,127.29,15157.24,3329.0,5
2,35000,35000,35000,13.08,797.80,6,1.0,125890.0,Fully Paid,debt_consolidation,VA,Jul-05,66.70%,16.53,35067.40,28648.0,6
3,17500,17500,17500,17.19,625.58,5,5.0,82000.0,Fully Paid,debt_consolidation,MO,Jun-06,83.60%,421.70,17955.12,11815.0,5
4,13750,13750,13750,28.80,574.71,4,NaN,17808.0,Fully Paid,credit_card,OH,Apr-90,87.60%,67.98,13861.98,2802.0,6


In [25]:
df1.drop(['revol_util','total_rec_int','earliest_cr_line','funded_amnt_inv'], axis=1, inplace=True)

In [26]:
df1.head()

,loan_amnt,funded_amnt,int_rate,installment,grade,emp_length,annual_inc,loan_status,purpose,addr_state,last_pymnt_amnt,avg_cur_bal,num_actv_rev_tl
0,10000,10000,12.40,224.48,6,NaN,38000.0,Fully Paid,credit_card,GA,10163.74,19266.0,2
1,15000,15000,17.97,380.66,4,9.0,45000.0,Fully Paid,debt_consolidation,FL,15157.24,3329.0,5
2,35000,35000,13.08,797.80,6,1.0,125890.0,Fully Paid,debt_consolidation,VA,35067.40,28648.0,6
3,17500,17500,17.19,625.58,5,5.0,82000.0,Fully Paid,debt_consolidation,MO,17955.12,11815.0,5
4,13750,13750,28.80,574.71,4,NaN,17808.0,Fully Paid,credit_card,OH,13861.98,2802.0,6


In [28]:
df1.to_csv('FullDataafterSplit&withclean.csv')